In [1]:
#データの読み込み
import numpy as np
import pandas as pd 
from config import Config

<head>
<h1>目次</h1>
</head>
<body>
<p><h4>1.データの抽出</h4></p>
<p><h4>2.データ品質の確認</h4><p>
<h1></h1>
</body>

<head>
    <b>1.データ抽出</b>
    <base href="C:\Users\dso-s.gao\Desktop￥" > 
</head>
<body>
    <p><b>目的：サンプル走行データの読み込み。pandasデータフレームにする</b></p>
    <hr />
    <p></p>
</body>

In [34]:
#dataの保存先絶対パスを取得
opt = Config()
data_path_aions = getattr(opt, "data_path_aions")
data_path_gs3 = getattr(opt, "data_path_gs3")
data_path_gs4 = getattr(opt, "data_path_gs4")

In [63]:
#走行データ読み込み。データフレームにする
def readtable(abspath, headerpath="gs3_head.txt"):
    
    """
    path:データの保存先の絶対パス
    """
    data = pd.read_table(abspath + "/all.txt", header=None)
    
    if headerpath:
        with open(abspath +  headerpath) as f:
            header = f.read()
            header = [i.strip() for i in header.split(",")]
        data.columns = header 
    return data 

In [55]:
with open(data_path_gs4 +  "gs4_head.txt") as f:
    header = f.read()
    header = [i.strip() for i in header.split(",")]

In [66]:
gs3_data = readtable(data_path_gs3) #gs3の車データを読み込み
gs4_data = readtable(data_path_gs4, headerpath="gs4_head.txt") #gs4の車データを読み込み

<head>
    <b>2.データ品質の確認</b>
    <base href="C:\Users\dso-s.gao\Desktop￥" > 
</head>
<body>
    <p><b>目的：サンプルデータの欠損率、異常値を確認</b></p>
    <hr />
    <p></p>
</body>